In [1]:
# Install Java, Spark, and Findspark
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!apt-get update
!apt-get install openjdk-8-jdk
!wget -q https://downloads.apache.org/spark/spark-2.4.6/spark-2.4.6-bin-hadoop2.7.tgz
!tar xf spark-2.4.6-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.6-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Ign:13 https://developer.download.nvidia.com/comp

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2020-08-15 22:41:58--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  3.61MB/s    in 0.2s    

2020-08-15 22:41:59 (3.61 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Big-Data-Project1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
#%pyspark
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
mobile_electronics_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Mobile_Electronics_v1_00.tsv.gz"), sep="\t", header=True)
mobile_electronics_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   20422322| R8MEA6IGAHO0B|B00MC4CED8|     217304173|BlackVue DR600GW-PMP|Mobile_Electronics|          5|            0|          0|   N|                Y|         Very Happy!|As advertised. Ev...| 2015-08-31|
|         US|   40835037|R31LOQ8JGLPRLK|B00OQMFG1Q|     137313254|GENSSI GSM / GPS ...|Mobile_Electronics|      

In [19]:
# the number of records in the data set
print(mobile_electronics_df.count())

104975


In [6]:
# Create Products DF design for table products
products_df = mobile_electronics_df.select("product_id", "product_title").dropDuplicates()
products_df = products_df.distinct()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B01075GRAO|ANYSECU QYT KT890...|
|B00QVXNV60|BRG AH2000G-ANR A...|
|B004ECO5VA|Holux Mini Smart ...|
|B00LGNKLXG|Aweek® Car Antenn...|
|B00M1Y87I0|      sentey ls 4063|
|B002ABX29I|Micca Slim Portab...|
|B00XXA2T2G|Centurion GT1 Car...|
|B00Q5A5PD6|HYUNDAI HCR-580 V...|
|B00VFEK8IY|Z ZTDM Mini Stere...|
|B00L85M8VU|Settonbrothers Ul...|
|B00SUXD5GU|[NEW RELEASE] JEB...|
|B00KJ7MVYW|Fincibo (TM) Appl...|
|B00A6YQQS0|Minisuit Aero Fol...|
|B009S99IYI|PIO TSG1644R 6.5 ...|
|B006PAZCXY|eForCity® Anti-Gl...|
|B00OXD61OE|SHE'O® 2x Cool No...|
|B000IBMI1K|RCA 200-Watt Mono...|
|B00UN6955M|Aisster(TM) Lot 1...|
|B00V684NCQ|LSD 8GB Flash Dri...|
|B00A609WOO|eForCity Hybrid S...|
+----------+--------------------+
only showing top 20 rows



In [7]:

#look at the schema of products_df
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [8]:
# Create Review df for table review_id
review_df = mobile_electronics_df.select("review_id", "customer_id", "product_id", "product_parent", "review_date").dropDuplicates()
review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R2GLAN4K93R537|   13007309|B00CJZLHN8|     869734493| 2015-08-29|
|R3FW20KZQ1ZBT7|   10557964|B012EDG356|     621314231| 2015-08-25|
|R25XXL935CX9M7|   28581092|B00BUFBQWU|     904426906| 2015-08-22|
|R18KB7EE7P34H4|   32375496|B00TDJ5VTO|     959170259| 2015-08-17|
|R2SMFL6Y5GWO41|   15988860|B00J46XO9U|     744008282| 2015-08-14|
| RDY61WDZNY97P|   19588345|B00MFM8AUI|     570563566| 2015-08-13|
|R2ALAWL75ID2JM|    2393847|B00SFJBPRK|     385604844| 2015-08-13|
|R233VN1EZXR72Q|    9096341|B00VU7PGPQ|     973149575| 2015-08-10|
|R3NWVGW2T1XDXF|   13500296|B009WQQYG2|     434298526| 2015-08-10|
|R3MDXVMIJWCQ4Y|   45834093|B00UF3PCTG|     103348577| 2015-08-10|
| RZ9NYUA1338J0|    5368734|B012EA8OB0|     702203341| 2015-08-06|
| RXYKD7EXJRP5G|   16330124|B00TDAY5ZE|     178040687| 2015-08

In [9]:
#look at the schema of review_df
from pyspark.sql.types import *
review_df = review_df.withColumn("customer_id2", review_df["customer_id"].cast(IntegerType())).drop("customer_id").withColumnRenamed("customer_id2", "customer_id")
review_df = review_df.withColumn("product_parent2", review_df["product_parent"].cast(IntegerType())).drop("product_parent").withColumnRenamed("product_parent2", "product_parent")
review_df = review_df.withColumn("review_date2", review_df["review_date"].cast(DateType())).drop("review_date").withColumnRenamed("review_date2", "review_date")
review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [10]:
customers_df = review_df.groupBy("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   30701952|             1|
|    1855744|             1|
|   52760451|             1|
|   20318398|             1|
|   10412939|             1|
|   10313569|             1|
|   17103239|             1|
|    1397686|             1|
|   44842505|             1|
|   16959619|             1|
|   26918067|             1|
|   33047049|             1|
|   26904397|             1|
|   11252502|             1|
|   47814156|             1|
|   14223449|             1|
|   32541548|             1|
|    9837980|             1|
|   38933600|             1|
|   22485819|             1|
+-----------+--------------+
only showing top 20 rows



In [11]:
customers_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [12]:
vines_df = mobile_electronics_df.select("review_id", "star_rating", "helpful_votes", "total_votes", "vine").dropDuplicates()
vines_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RJWZF0T3DQHIP|          5|            2|          2|   N|
|R2PESGM2VMZR5A|          5|            0|          0|   N|
|R18MHHFKCQSFQP|          4|            0|          0|   N|
|R3G4SXGOVSSJC1|          5|            0|          0|   N|
|R3ONOFLYVUS1HF|          4|            0|          0|   N|
| RTV1X5S22DPAK|          4|            1|          1|   N|
|R3VSHMUMY8JCFF|          5|            0|          0|   N|
|R3COVWM6R99QLG|          5|            0|          0|   N|
|R2RKRLW02UNOTX|          5|            0|          0|   N|
| RQBBWPWFFAGA9|          4|            0|          0|   N|
|R1DOGMETZK7IR5|          2|            1|          2|   N|
|R17P3B259KV0IJ|          5|            0|          0|   N|
|R3VNG4QIXWEBVW|          5|            0|          0|   N|
|R1A0NIYJ19ZQ7G|          1|            

In [13]:
vines_df = vines_df.withColumn("star_rating2", vines_df["star_rating"].cast(IntegerType())).drop("star_rating").withColumnRenamed("star_rating2", "star_rating")
vines_df = vines_df.withColumn("helpful_votes2", vines_df["helpful_votes"].cast(IntegerType())).drop("helpful_votes").withColumnRenamed("helpful_votes2", "helpful_votes")
vines_df = vines_df.withColumn("total_votes2", vines_df["total_votes"].cast(IntegerType())).drop("total_votes").withColumnRenamed("total_votes2", "total_votes")
vines_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- vine: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)



In [14]:
mode = "append"
jdbc_url="jdbc:postgresql://big-data-homework-db.cac8viaacwex.us-east-2.rds.amazonaws.com:5432/amazon_review"
config = {"user":"<user>", 
          "password": "<password>", 
          "driver":"org.postgresql.Driver"}

In [15]:
# Write DataFrame to products table in RDS

products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [16]:
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [17]:
review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [18]:
vines_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)